In [1]:
import requests
import folium
import pandas as pd
from dotenv import load_dotenv
import os
import googlemaps
from geopy.distance import great_circle
import time

load_dotenv()
API_KEY = os.getenv('GOOGLE_API_KEY')
gmaps = googlemaps.Client(key=API_KEY)

#### This is here in case we make changes to the dataset, it will be easier to pull in the data. Your comp won't be able to run this.
#### COMMENT OUT ####
# import gspread
# import json
# google_cred = os.getenv('GOOGLE_CRED')
# gc = gspread.service_account_from_dict(json.loads(google_cred.replace('"', "'").replace("'", '"')))
# tam_wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/13iInhOybCj3FmACmbuAqXgsIzppdrghgluNpqa0sQCg/edit#gid=1807133386')
# school_df = pd.DataFrame(tam_wb.worksheet('Target Prospects').get_all_records())
# customer_df = pd.DataFrame(tam_wb.worksheet('Best Customers').get_all_records())
#####################

# Golf Course

In [2]:
# Load golf course data
golf_course_df = pd.read_csv('../data/golf_top100.csv')
golf_courses = list(golf_course_df['Course'])

In [3]:
# Get coordinates for each golf course
coordinates = {}

for i in range(len(golf_course_df)):
    params = {
        'address': golf_course_df['Course'][i],
        'key': API_KEY
    }
    response = requests.get('https://maps.googleapis.com/maps/api/geocode/json', params=params)
    json_data = response.json()
    if json_data['results']:
        location = json_data['results'][0]['geometry']['location']
        golf_course_df.loc[i, 'lat'] = location['lat']
        golf_course_df.loc[i, 'long'] = location['lng']
    else:
        print(f"No results for {golf_course_df['Course'][i]}")
        print(json_data)
        golf_course_df.loc[i, 'lat'] = ''
        golf_course_df.loc[i, 'long'] = ''
   
    # Clean up the names ect.
    golf_course_df.loc[i, 'Course'] = golf_course_df.loc[i, 'Course'].title()
    golf_course_df.loc[i, 'City'] = golf_course_df.loc[i, 'Course'].split(',')[1].strip()
    golf_course_df.loc[i, 'State'] = golf_course_df.loc[i, 'Course'].split(',')[2].strip()
    golf_course_df.loc[i, 'Course'] = golf_course_df.loc[i, 'Course'].split(',')[0].strip()

In [4]:
# # concatenate all long name values together to get the full street address
# json_data['results'][0]['address_components']

In [5]:
golf_course_df.head()

,Ranking,Course,Price,lat,long,City,State
0,1,Pebble Beach Golf Links,$600,36.569655,-121.949755,Pebble Beach,Calif
1,2,Bandon Dunes Golf Resort,$350,43.193618,-124.389932,Pacific Dunes,Ore
2,3,Pinehurst Resort (No 2),$525,35.195790,-79.473674,Pinehurst,Nc
3,4,Bethpage Black Course,$97,40.742865,-73.455077,Farmingdale,Ny
4,5,Whistling Straits Golf Club (Straits),$500,43.851058,-87.735142,Kohler,Wisc


# School

In [6]:
# Load school data
school_df = pd.read_csv('../data/target_prospects.csv')

# Drop any rows without an address
school_df = school_df.replace('', None).dropna(subset=['Street Address']).reset_index(drop=True)

In [7]:
school_df['lat'] = ''
school_df['long'] = ''

for i in range(len(school_df)):
    try:
        # Try just the street address
        school_df.loc[i, 'lat'] = gmaps.geocode(school_df['Street Address'][i])[0]['geometry']['location']['lat']
        school_df.loc[i, 'long'] = gmaps.geocode(school_df['Street Address'][i])[0]['geometry']['location']['lng']
    except IndexError:
        try: 
            # Try address with state
            school_df.loc[i, 'lat'] = gmaps.geocode(school_df['Street Address'][i] + ', ' + school_df['State/Region'][i])[0]['geometry']['location']['lat']
            school_df.loc[i, 'long'] = gmaps.geocode(school_df['Street Address'][i] + ', ' + school_df['State/Region'][i])[0]['geometry']['location']['lng']
            # update street address to include state for future use
            school_df.loc[i, 'Street Address'] = school_df['Street Address'][i] + ', ' + school_df['State/Region'][i]
        except IndexError:
            try:
                # Try school name
                school_df.loc[i, 'lat'] = gmaps.geocode(school_df['Company name'][i])[0]['geometry']['location']['lat']
                school_df.loc[i, 'long'] = gmaps.geocode(school_df['Company name'][i])[0]['geometry']['location']['lng']
            except IndexError:
                print(f"No results for {school_df['Company name'][i]} at {school_df['Street Address'][i]}")
                school_df.loc[i, 'lat'] = ''
                school_df.loc[i, 'long'] = ''
    
    ## Look for duplicate addresses - sites & corporate offices

In [8]:
# Remove any where the address could not be found
school_df = school_df.replace('', None).dropna(subset=['lat']).reset_index(drop=True)

In [9]:
school_df

,Record ID,Company name,Status,Record Type,New ICP,Reason not ICP 1,Segment,School Type (picklist),Total Students,Number of child companies,LMS (multi-select),SIS (multi-select),Street Address,State/Region,lat,long
0,4104332334,Advanced College - Corporate Office,Target Account,Corporate Office,ICP 1,ICP 1,Allied Health,Private for-profit,486,2.0,Canvas,RGM,"13180 Paramount Boulevard, California",California,33.915372,-118.157844
1,4108671175,Alaska Career College - Anchorage,Target Account,Lone Campus,ICP 1,ICP 1,Allied Health,Private for-profit,412,NaN,Canvas,CVUE,"1415 E Tudor Road Anchorage, AK",Alaska,61.18138,-149.85694
2,4108718586,American Career College - Corporate Office,Target Account,Corporate Office,ICP 1,ICP 1,Allied Health,Private for-profit,"6,349",3.0,Canvas; Blackboard,CVUE,1200 N. Magnolia Ave.,California,33.849889,-117.975558
3,4104294015,American College of Education - Indianapolis,Target Account,Lone Campus,ICP 1,ICP 1,Allied Health,Private for-profit,"8,112",NaN,Canvas,Anthology,"101 West Ohio Street Suite 1200 Indianapolis, IN",Indiana,39.769698,-86.160226
4,6920844630,American College of Healthcare and Technology ...,Target Account,Corporate Office,ICP 1,ICP 1,Allied Health,Private for-profit,621,3.0,Moodle,CVUE,11801 Pierce Street,California,40.772366,-73.517001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,4108753843,Platt College - San Diego,Not Viable,Franchise,Not Viable,Not Viable,Career + Trade,Private for-profit,96,0.0,NaN,NaN,"6250 El Cajon Blvd San Diego, CA",California,32.763942,-117.06364
562,6936344884,Toni & Guy Hairdressing Academy - Atlanta,Not Viable,Franchise,Not Viable,Not Viable,Beauty + Wellness,Private for-profit,65,0.0,PivotPoint,RGM,"1355 Roswell Rd - Marietta, GA 30062",GA,33.952494,-84.515173
563,7529824352,Toni & Guy Hairdressing Academy - Jacksonville,Not Viable,Franchise,Not Viable,Not Viable,Beauty + Wellness,NaN,NaN,0.0,PivotPoint,NaN,"8206 Philips Hwy. Suite 33 - Jacksonville, FL ...",FL,30.219237,-81.586595
564,7876109240,Toni & Guy Hairdressing Academy - Modesto,Not Viable,Franchise,Not Viable,Not Viable,NaN,NaN,NaN,0.0,PivotPoint,NaN,"2425 McHenry Ave - Modesto, CA 95350",CA,37.677238,-120.995474


# Create Map

In [10]:
m = folium.Map(location=[39.8283, -98.5795], zoom_start=4)

In [11]:
# Add purple markers for each school
for i in range(len(school_df)):
    pop_up_data = f"<strong>School: </strong>{school_df['Company name'][i]}<br><br><strong>School Type: </strong>{school_df['School Type (picklist)'][i]}<br><br><strong>School Type: </strong>{school_df['Segment'][i]}"
    folium.Marker(
        location=[school_df['lat'][i], school_df['long'][i]], 
        popup=pop_up_data,
        icon=folium.Icon(color='purple')
    ).add_to(m)

In [12]:
# Add blue markers for each golf course
for i in range(len(golf_course_df)):
    pop_up_data = f"<strong>Course: </strong>{golf_course_df['Course'][i]}<br><br><strong>Location: </strong>{golf_course_df['City'][i]}, {golf_course_df['State'][i]}"
    folium.Marker(
        location=[golf_course_df['lat'][i], golf_course_df['long'][i]], 
        popup=pop_up_data,
        icon=folium.Icon(color='green')
    ).add_to(m)

In [13]:
m

In [14]:
# Display the map
m.save('map.html')

In [15]:
# Find all schools within 100 miles of each golf course using as the crow flies distance
distance_data = []

for i in range(len(golf_course_df)):
    for j in range(len(school_df)):
        golf_course = (golf_course_df['lat'][i], golf_course_df['long'][i])
        school = (school_df['lat'][j], school_df['long'][j])
        distance = great_circle(golf_course, school).miles

        if distance <= 100:
            distance_data.append({
                'golf_course': golf_course_df['Course'][i],
                'school': school_df['Company name'][j],
                'golf_course_geo': golf_course,
                'school_geo': school,
                'school_address': school_df['Street Address'][j],
                'crow_distance': distance
            })

distance_df = pd.DataFrame(distance_data)

In [16]:
distance_df

,golf_course,school,golf_course_geo,school_geo,school_address,crow_distance
0,Pebble Beach Golf Links,Academy of Art University - San Francisco,"(36.5696553, -121.9497555)","(37.7879584, -122.4004678)","79 New Montgomery San Francisco, CA",87.756983
1,Pebble Beach Golf Links,Cinta Aveda,"(36.5696553, -121.9497555)","(37.7909576, -122.4042735)",305 Kearny Street,88.014954
2,Pebble Beach Golf Links,Contra Costa Medical Career College - Antioch,"(36.5696553, -121.9497555)","(37.97679979999999, -121.8015862)","4041 Lone Tree Way Antioch, CA",97.565091
3,Pebble Beach Golf Links,Five Branches University - Santa Cruz,"(36.5696553, -121.9497555)","(36.9644346, -121.9980494)","200 7th Avenue Santa Cruz, CA",27.407312
4,Pebble Beach Golf Links,National Holistic Institute - Emeryville,"(36.5696553, -121.9497555)","(37.8418091, -122.2878972)","5900 Doyle Street Emeryville, CA",89.845364
...,...,...,...,...,...,...
1049,Grayhawk Golf Club (Talon),Empire Beauty School - NW Phoenix,"(33.6780969, -111.8971717)","(33.5380591, -112.1181902)","2727 W. Glendale Ave. #200 Phoenix, AZ",15.980411
1050,Grayhawk Golf Club (Talon),Empire Beauty Schools - Chandler-Gilbert,"(33.6780969, -111.8971717)","(33.3475191, -111.8603127)","2978 N Alma School Rd #1-3 Chandler, AZ",22.939236
1051,Grayhawk Golf Club (Talon),Toni & Guy Hairdressing Academy - Scottsdale,"(33.6780969, -111.8971717)","(33.5367299, -111.8892534)","8939 E Talking Stick Way - Scottsdale, AZ 85250",9.778152
1052,Grayhawk Golf Club (Talon),West-MEC,"(33.6780969, -111.8971717)","(33.5150969, -112.271793)",5487 North 99th Avenue,24.324336


In [19]:
# # Use the google sheets API to get the driving distance and time between each school and golf course

# for i in range(len(distance_df)):
#     origin = distance_df['school_address'][i]
#     destination = distance_df['golf_course'][i]

#     url = f"https://maps.googleapis.com/maps/api/distancematrix/json?origins={origin}&destinations={destination}&units=imperial&key={API_KEY}"

#     response = requests.get(url)
#     data = response.json()

#     if data['status'] == 'REQUEST_DENIED':
#         print(f"{i} rows have been processed.")
#         print("API limit, wait")
#         time.sleep(20)
#     elif data['rows'][0]['elements'][0]['status'] == 'NOT_FOUND':
#         print(f'{origin} and {destination} - No results found')
#     else:
#         distance = data['rows'][0]['elements'][0]['distance']['text']
#         duration = data['rows'][0]['elements'][0]['duration']['text']

#         distance_df.loc[i, 'distance'] = distance
#         distance_df.loc[i, 'duration'] = duration

# # load the data into a csv file
# distance_df.to_csv('../data/distance_data.csv')

In [17]:
# read in CSV if you don't want to run the whole thing
distance_df = pd.read_csv('../data/distance_data.csv').drop(columns=['Unnamed: 0'])

In [18]:
def convert_time_to_minutes(time):
    '''Convert 2 hours 5 mins to 125 minutes and 55 mins to 55 minutes'''
    if time != time:
        return time
    if len(time.split(' ')) == 4:
        return int(time.split(' ')[0]) * 60 + int(time.split(' ')[2])
    else:
        return int(time.split(' ')[0])


In [19]:
# convert distance to float
distance_df['distance_mi'] = distance_df['distance'].apply(lambda x: float(str(x).split(' ')[0]))
# convert duration to minutes
distance_df['duration_min'] = distance_df['duration'].apply(lambda x: (convert_time_to_minutes(x)))

In [20]:
distance_df

,golf_course,school,golf_course_geo,school_geo,school_address,crow_distance,distance,duration,distance_mi,duration_min
0,Pebble Beach Golf Links,Academy of Art University - San Francisco,"(36.5696553, -121.9497555)","(37.7879584, -122.4004678)","79 New Montgomery San Francisco, CA",87.756983,124 mi,2 hours 5 mins,124.0,125.0
1,Pebble Beach Golf Links,Cinta Aveda,"(36.5696553, -121.9497555)","(37.7909576, -122.4042735)",305 Kearny Street,88.014954,124 mi,2 hours 11 mins,124.0,131.0
2,Pebble Beach Golf Links,Contra Costa Medical Career College - Antioch,"(36.5696553, -121.9497555)","(37.97679979999999, -121.8015862)","4041 Lone Tree Way Antioch, CA",97.565091,143 mi,2 hours 22 mins,143.0,142.0
3,Pebble Beach Golf Links,Five Branches University - Santa Cruz,"(36.5696553, -121.9497555)","(36.9644346, -121.9980494)","200 7th Avenue Santa Cruz, CA",27.407312,45.8 mi,55 mins,45.8,55.0
4,Pebble Beach Golf Links,National Holistic Institute - Emeryville,"(36.5696553, -121.9497555)","(37.8418091, -122.2878972)","5900 Doyle Street Emeryville, CA",89.845364,120 mi,2 hours 4 mins,120.0,124.0
...,...,...,...,...,...,...,...,...,...,...
1054,Grayhawk Golf Club (Talon),Empire Beauty School - NW Phoenix,"(33.6780969, -111.8971717)","(33.5380591, -112.1181902)","2727 W. Glendale Ave. #200 Phoenix, AZ",15.980411,NaN,NaN,NaN,NaN
1055,Grayhawk Golf Club (Talon),Empire Beauty Schools - Chandler-Gilbert,"(33.6780969, -111.8971717)","(33.3475191, -111.8603127)","2978 N Alma School Rd #1-3 Chandler, AZ",22.939236,NaN,NaN,NaN,NaN
1056,Grayhawk Golf Club (Talon),Toni & Guy Hairdressing Academy - Scottsdale,"(33.6780969, -111.8971717)","(33.5367299, -111.8892534)","8939 E Talking Stick Way - Scottsdale, AZ 85250",9.778152,11.1 mi,15 mins,11.1,15.0
1057,Grayhawk Golf Club (Talon),West-MEC,"(33.6780969, -111.8971717)","(33.5150969, -112.271793)",5487 North 99th Avenue,24.324336,32.9 mi,33 mins,32.9,33.0


# Choosing Golf Courses

In [21]:
deg_central_df = pd.read_csv('../data/degree_centrality.csv')
deg_central_df

,name,degree
0,Trump Golf Links At Ferry Point,22.883333
1,Bulle Rock,18.083333
2,Cog Hill (No. 4),15.250000
3,Atlantic City Country Club,15.166667
4,The Glen Club,14.250000
...,...,...
70,Bay Harbor Golf Club (Links/Quarry),0.250000
71,Omni Homestead (Cascades),0.250000
72,Taconic Golf Club,0.250000
73,The Links At Lighthouse Sound,0.250000


In [22]:
########## JOIN DATASETS ###########
distance_lim_df = distance_df[['golf_course', 'school', 'distance_mi', 'duration_min']] # only include the columns we need
# filter distance_df to only include the schools that are on the degree centrality list
school_course_df = distance_lim_df.merge(deg_central_df, left_on='golf_course', right_on='name', how='inner').drop(columns=['name'])

In [23]:
school_course_df

,golf_course,school,distance_mi,duration_min,degree
0,Pebble Beach Golf Links,Academy of Art University - San Francisco,124.0,125.0,2.550000
1,Pebble Beach Golf Links,Cinta Aveda,124.0,131.0,2.550000
2,Pebble Beach Golf Links,Contra Costa Medical Career College - Antioch,143.0,142.0,2.550000
3,Pebble Beach Golf Links,Five Branches University - Santa Cruz,45.8,55.0,2.550000
4,Pebble Beach Golf Links,National Holistic Institute - Emeryville,120.0,124.0,2.550000
...,...,...,...,...,...
993,Grayhawk Golf Club (Talon),Empire Beauty School - NW Phoenix,NaN,NaN,5.166667
994,Grayhawk Golf Club (Talon),Empire Beauty Schools - Chandler-Gilbert,NaN,NaN,5.166667
995,Grayhawk Golf Club (Talon),Toni & Guy Hairdressing Academy - Scottsdale,11.1,15.0,5.166667
996,Grayhawk Golf Club (Talon),West-MEC,32.9,33.0,5.166667


In [64]:
choose_course_df = school_course_df
# .sort_values(['degree', 'duration_min'], ascending=[False, False]).drop_duplicates(subset=['school'], keep='first').reset_index(drop=True)

In [65]:
choose_course_df

,golf_course,school,distance_mi,duration_min,degree
0,Pebble Beach Golf Links,Academy of Art University - San Francisco,124.0,125.0,2.550000
1,Pebble Beach Golf Links,Cinta Aveda,124.0,131.0,2.550000
2,Pebble Beach Golf Links,Contra Costa Medical Career College - Antioch,143.0,142.0,2.550000
3,Pebble Beach Golf Links,Five Branches University - Santa Cruz,45.8,55.0,2.550000
4,Pebble Beach Golf Links,National Holistic Institute - Emeryville,120.0,124.0,2.550000
...,...,...,...,...,...
993,Grayhawk Golf Club (Talon),Empire Beauty School - NW Phoenix,NaN,NaN,5.166667
994,Grayhawk Golf Club (Talon),Empire Beauty Schools - Chandler-Gilbert,NaN,NaN,5.166667
995,Grayhawk Golf Club (Talon),Toni & Guy Hairdressing Academy - Scottsdale,11.1,15.0,5.166667
996,Grayhawk Golf Club (Talon),West-MEC,32.9,33.0,5.166667


In [66]:
choose_grouped = choose_course_df.groupby(['golf_course', 'degree']).agg({'school': 'count', 'distance_mi':'mean', 'duration_min':'mean'}).sort_values(by='degree', ascending=False).reset_index()
choose_grouped = choose_grouped.merge(golf_course_df, left_on='golf_course', right_on='Course', how='inner').drop(columns=['Course','Ranking', 'lat', 'long'])
choose_grouped['Price'] = choose_grouped['Price'].fillna(0).apply(lambda x: int(str(x).replace('$', '').replace(',', '')))
choose_grouped = choose_grouped.rename(columns={'school': 'num_schools', 'distance_mi': 'avg_distance_mi', 'duration_min': 'avg_duration_min'})
# choose_grouped = choose_grouped[~choose_grouped['golf_course'].str.contains('Trump')].reset_index(drop=True) # dropping the Trump course to avoid politics
choose_grouped.head()

,golf_course,degree,num_schools,avg_distance_mi,avg_duration_min,Price,City,State
0,Trump Golf Links At Ferry Point,22.883333,52,53.262745,64.058824,237,Bronx,Ny
1,Bulle Rock,18.083333,46,75.288636,86.545455,100,Havre De Grace,Md
2,Cog Hill (No. 4),15.250000,25,51.096000,57.560000,155,Lemont,Il
3,Atlantic City Country Club,15.166667,36,89.434286,92.485714,100,Northfield,Nj
4,The Glen Club,14.250000,26,48.346154,55.038462,200,Glenview,Ill


In [67]:
choose_grouped['attendees'] = round(choose_grouped['num_schools'] * 2 * .1) # assume 35% attendance when inviting 2 people from each school
choose_grouped['cost'] = choose_grouped['attendees'] * choose_grouped['Price'] 
choose_grouped = choose_grouped.sort_values(['degree', 'Price', 'attendees'], ascending=[False, True, False])
# choose_grouped = choose_grouped[choose_grouped['Price'] < 200].reset_index(drop=True) # remove any super expensive golf courses
choose_grouped = choose_grouped[choose_grouped['num_schools'] > 5].reset_index(drop=True) # must be catering to at least 5 schools
choose_grouped['total'] = choose_grouped['cost'].cumsum()
chosen_courses = choose_grouped[choose_grouped['total'] <= 10000].reset_index(drop=True)
chosen_courses

,golf_course,degree,num_schools,avg_distance_mi,avg_duration_min,Price,City,State,attendees,cost,total
0,Trump Golf Links At Ferry Point,22.883333,52,53.262745,64.058824,237,Bronx,Ny,10.0,2370.0,2370.0
1,Bulle Rock,18.083333,46,75.288636,86.545455,100,Havre De Grace,Md,9.0,900.0,3270.0
2,Cog Hill (No. 4),15.250000,25,51.096000,57.560000,155,Lemont,Il,5.0,775.0,4045.0
3,Atlantic City Country Club,15.166667,36,89.434286,92.485714,100,Northfield,Nj,7.0,700.0,4745.0
4,The Glen Club,14.250000,26,48.346154,55.038462,200,Glenview,Ill,5.0,1000.0,5745.0
5,Bethpage Black Course,14.216667,43,59.042857,73.285714,97,Farmingdale,Ny,9.0,873.0,6618.0
6,Pelican Hill Golf Club (Ocean South),12.466667,34,50.431250,54.718750,280,Newport Coast,Calif,7.0,1960.0,8578.0
7,Arnold Palmer’S Bay Hill Club (Champion),12.264286,25,61.476000,69.560000,220,Orlando,Fla,5.0,1100.0,9678.0


In [72]:
# chosen_courses.to_csv('../data/chosen_courses.csv')
chosen_courses = pd.read_csv('../data/chosen_courses.csv').drop(columns=['Unnamed: 0'])

In [73]:
chosen_courses

,golf_course,degree,num_schools,avg_distance_mi,avg_duration_min,Price,City,State,attendees,cost,total
0,Trump Golf Links At Ferry Point,23.883333,52,53.262745,64.058824,237,Bronx,Ny,10.0,2370.0,2370.0
1,Bulle Rock,21.083333,29,67.617857,82.392857,100,Havre De Grace,Md,6.0,600.0,2970.0
2,The Glen Club,15.500000,26,48.346154,55.038462,200,Glenview,Ill,5.0,1000.0,3970.0
3,Streamsong Resort (Red),13.680952,25,82.496000,93.480000,300,Streamsong,Fla,5.0,1500.0,5470.0
4,Reynolds Lake Oconee (Oconee),13.083333,22,89.213636,91.909091,200,Greensboro,Ga,4.0,800.0,6270.0
5,Pelican Hill Golf Club (Ocean South),12.966667,34,50.431250,54.718750,280,Newport Coast,Calif,7.0,1960.0,8230.0
6,Lake Of Isles (North),10.516667,12,78.950000,83.583333,150,North Stonington,Conn,2.0,300.0,8530.0
7,Trump National Doral,9.500000,21,34.114286,42.238095,270,(Blue Monster) Miami,Fla,4.0,1080.0,9610.0


In [74]:
chosen_courses['num_schools'].sum()

221

In [79]:
# filter choose_course_df for only the chosen courses
chosen_schools = choose_course_df[choose_course_df['golf_course'].isin(chosen_courses['golf_course'])].reset_index(drop=True)
chosen_schools = chosen_schools.sort_values(['golf_course', 'duration_min']).reset_index(drop=True)
chosen_schools = chosen_schools[['school', 'golf_course', 'distance_mi', 'duration_min']]
chosen_schools = chosen_schools.drop_duplicates(subset=['school'], keep='first').reset_index(drop=True)
# chosen_schools.to_csv('../data/chosen_schools.csv')
chosen_schools

,school,golf_course,distance_mi,duration_min
0,Aveda Institute - Maryland,Bulle Rock,15.5,28.0
1,Dawn Career Institute LLC - Newark,Bulle Rock,30.9,38.0
2,All-State Career School - Corporate Office,Bulle Rock,34.6,39.0
3,Empire Beauty Schools - Glen Burnie-Baltimore,Bulle Rock,41.1,47.0
4,Empire Beauty Schools - Owings Mills-Baltimore,Bulle Rock,44.0,51.0
...,...,...,...,...
216,International Institute For Health Care Profes...,Trump National Doral,55.0,63.0
217,Cambridge College of Healthcare & Technology -...,Trump National Doral,56.9,64.0
218,The Salon Professional Academy - Delray Beach,Trump National Doral,58.4,66.0
219,Empire Beauty Schools - West Palm Beach,Trump National Doral,75.0,78.0


# Create Map of Chosen Golf Courses and Schools

In [114]:
chosen_school_address = school_df[school_df['Company name'].isin(chosen_schools['school'])].reset_index(drop=True)

In [115]:
# Correcting a wrong address
chosen_school_address.loc[chosen_school_address['Company name'] == 'Woodruff Medical Training and Testing - Corporate Office', 'lat'] = '33.84015'
chosen_school_address.loc[chosen_school_address['Company name'] == 'Woodruff Medical Training and Testing - Corporate Office', 'long'] = '-84.23897'

In [116]:
# Dropping duplicate that has the wrong address
chosen_school_address[chosen_school_address['Company name'] == 'Aveda Institute - South Florida']
chosen_school_address = chosen_school_address.drop(157).reset_index(drop=True)

In [117]:
chosen_school_address

,Record ID,Company name,Status,Record Type,New ICP,Reason not ICP 1,Segment,School Type (picklist),Total Students,Number of child companies,LMS (multi-select),SIS (multi-select),Street Address,State/Region,lat,long
0,4108718586,American Career College - Corporate Office,Target Account,Corporate Office,ICP 1,ICP 1,Allied Health,Private for-profit,"6,349",3.0,Canvas; Blackboard,CVUE,1200 N. Magnolia Ave.,California,33.849889,-117.975558
1,6920844630,American College of Healthcare and Technology ...,Target Account,Corporate Office,ICP 1,ICP 1,Allied Health,Private for-profit,621,3.0,Moodle,CVUE,11801 Pierce Street,California,40.772366,-73.517001
2,4108718708,Automotive Training Center - Corporate Office,Target Account,Corporate Office,ICP 1,ICP 1,Career + Trade,Private for-profit,738,2.0,D2L,CVUE,114 Pickering Way,Pennsylvania,40.058266,-75.643175
3,4104344606,Aveda - Fort Lauderdale,Prospect,Franchise,ICP 1,ICP 1,Beauty + Wellness,Private for-profit,506,NaN,PivotPoint,FAME,"495 N Semoran Blvd, Ste 6",Florida,28.604045,-81.305642
4,4108753240,Aveda Institute - Tampa Bay,Prospect,Franchise,ICP 1,ICP 1,Beauty + Wellness,Private for-profit,780,NaN,PivotPoint,FAME,"28272 US Highway 19 N - Clearwater, Florida",Florida,28.029168,-82.739013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,6936265106,Tricoci - Janesville,Prospect,Franchise,ICP 4,Too Small,Beauty + Wellness,Private for-profit,56,0.0,PivotPoint,FAME,"2310 W. Court Street Janesville, WI",Wisconsin,42.679766,-89.048915
217,4108754052,Wisconsin Academy a Paul Mitchell Partner Group,Prospect,Franchise,ICP 4,Too Small,Beauty + Wellness,Private for-profit,44,NaN,NaN,FAME,"611 E Porter St, Wisconsin",Wisconsin,43.189559,-88.989763
218,4108647926,Woodruff Medical Training and Testing - Corpor...,Target Account,Corporate Office,ICP 4,Too Small,Allied Health,Private for-profit,56,2.0,Canvas,Campus Cafe,1927 Lakeside Parkway,Georgia,33.84015,-84.23897
219,4108753843,Platt College - San Diego,Not Viable,Franchise,Not Viable,Not Viable,Career + Trade,Private for-profit,96,0.0,NaN,NaN,"6250 El Cajon Blvd San Diego, CA",California,32.763942,-117.06364


In [118]:
chosen_course_address = golf_course_df[golf_course_df['Course'].isin(chosen_courses['golf_course'])].reset_index(drop=True)

In [119]:
m = folium.Map(location=[39.8283, -98.5795], zoom_start=4)

In [120]:
# Add purple markers for each school
for i in range(len(chosen_school_address)):
    pop_up_data = f"<strong>School: </strong>{chosen_school_address['Company name'][i]}<br><br><strong>School Type: </strong>{chosen_school_address['School Type (picklist)'][i]}<br><br><strong>School Type: </strong>{chosen_school_address['Segment'][i]}"
    folium.Marker(
        location=[chosen_school_address['lat'][i], chosen_school_address['long'][i]], 
        popup=pop_up_data,
        icon=folium.Icon(color='purple')
    ).add_to(m)

In [121]:
# Add blue markers for each golf course
for i in range(len(chosen_course_address)):
    pop_up_data = f"<strong>Course: </strong>{chosen_course_address['Course'][i]}<br><br><strong>Location: </strong>{chosen_course_address['City'][i]}, {chosen_course_address['State'][i]}"
    folium.Marker(
        location=[chosen_course_address['lat'][i], chosen_course_address['long'][i]], 
        popup=pop_up_data,
        icon=folium.Icon(color='green')
    ).add_to(m)

In [122]:
m